In [26]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Efecto del Tracking Escolar y Inferencia con Muestras Agrupadas

En contextos donde las unidades de observación están naturalmente agrupadas (como estudiantes
dentro de escuelas), la independencia entre observaciones puede no cumplirse. Esto afecta la inferencia
estadística, ya que los errores estándar convencionales pueden subestimar la varianza real
del estimador. En este ejercicio trabajaremos con datos de un experimento aleatorizado en escuelas
de Kenia (Duflo, Dupas y Kremer, 2011), que evaluó el efecto de dividir a los estudiantes en clases
según su nivel de rendimiento inicial (“tracking”).
### Objetivo
Replicar la estimación del efecto promedio del tracking sobre el rendimiento escolar con inferencia
adecuada para datos agrupados.
### Base de datos
Utilice el archivo DDK2011.dta. La variable dependiente es el puntaje estandarizado TestScore, y
la variable de tratamiento es Tracking, que indica si la escuela aplicó tracking.

In [27]:
url=r"C:\Users\Camil\Documents\GitHub\ECOP2037_CE\hmw2\DDK2011.dta"
df = pd.read_stata(url)

## Parte I: Estimación básica
1. Estime el siguiente modelo por Mínimos Cuadrados Ordinarios (OLS):
$$
\text{TestScore}_{ig} = \alpha + \gamma \text{Tracking}_g + e_{ig}
$$

2. Reporte el valor de $\hat{\gamma}$ y su error estándar usando:
(a) Errores estándar convencionales (homocedásticos).

In [28]:
y=df['totalscore']
x=df['tracking']
x = sm.add_constant(x)  
model_conventional = sm.OLS(y, x).fit()
hat_gamma=model_conventional.params[1]
hat_se=model_conventional.bse[1]
print(f'Gamma estimado=', hat_gamma)
print(f'Error estandar estimado=', hat_se)

Gamma estimado= 1.2581967566352972
Error estandar estimado= 0.2389277353144617


C:\Users\Camil\AppData\Local\Temp\ipykernel_3004\3137233708.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_gamma=model_conventional.params[1]
C:\Users\Camil\AppData\Local\Temp\ipykernel_3004\3137233708.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se=model_conventional.bse[1]


(b) Errores estándar robustos agrupados por escuela.

In [29]:
model_clustered = sm.OLS(y, x).fit( cov_type='cluster', cov_kwds={'groups': df['schoolid']})
hat_gamma1=model_clustered.params[1]
hat_se1=model_clustered.bse[1]
print(f'Gamma estimado=', hat_gamma1)
print(f'Error estandar estimado=', hat_se1)

Gamma estimado= 1.2581967566352972
Error estandar estimado= 0.7037256469657598


C:\Users\Camil\AppData\Local\Temp\ipykernel_3004\812076604.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_gamma1=model_clustered.params[1]
C:\Users\Camil\AppData\Local\Temp\ipykernel_3004\812076604.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se1=model_clustered.bse[1]


3. Comente las diferencias. ¿Qué implicancias tiene para la significancia estadística del efecto estimado?

El error estándar convencional es 0.24, mientras que el error estándar clustered por escuela es 0.70. Esto muestra que al no tener en cuenta la agrupación por escuela,es decir, que los estudiantes de una misma escuela pueden estar correlacionados, el error estándar se subestima. Como resultado, el efecto de Tracking parece ser estadísticamente significativo con errores convencionales, pero pierde significancia cuando se usan errores agrupados. 

## Parte II: Desafíos adicionales
### Challenge 1: Robustez con controles individuales
Agregue las siguientes variables de control al modelo anterior:
- Edad del estudiante.
- Género.
- Puntaje inicial.


¿Cambia la magnitud o significancia de $\hat{\gamma}$? ¿Por qué?

In [ ]:
##FALTA AGREGAR PUNTAJE INICIAL 
model_clustered2 = smf.ols('totalscore ~ tracking + agetest + girl ', data=df).fit()
hat_gamma2=model_clustered2.params[1]
hat_se2=model_clustered2.bse[1]
print(model_clustered2.summary())

                            OLS Regression Results                            
Dep. Variable:             totalscore   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     17.72
Date:                Wed, 11 Jun 2025   Prob (F-statistic):           1.91e-11
Time:                        22:38:50   Log-Likelihood:                -20873.
No. Observations:                5760   AIC:                         4.175e+04
Df Residuals:                    5756   BIC:                         4.178e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.7389      0.813     11.984      0.0

C:\Users\Camil\AppData\Local\Temp\ipykernel_3004\2737613133.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_gamma2=model_clustered2.params[1]
C:\Users\Camil\AppData\Local\Temp\ipykernel_3004\2737613133.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se2=model_clustered2.bse[1]


## Challenge 2: Heterogeneidad del efecto del tracking
Cree una variable BottomHalf que sea 1 si el estudiante estaba en la mitad inferior del puntaje
inicial en su escuela. Estime:
$$
\text{TestScore}_{ig} = \alpha + \gamma_1  \text{Tracking}_g + \gamma_2 \text{BottomHalf}_ig + \gamma_3 \text{Tracking}_g * \text{BottomHalf}_ig + e_{ig}
$$


Interprete el coeficiente $\hat{\gamma_3}$
¿Es diferente el efecto del tracking para estudiantes de menor rendimiento
inicial?

In [31]:
## crear variable bottomhalf
tracking_bottomhalf=df['tracking']*df['bottomhalf']
model_clustered3 = smf.ols('totalscore ~ tracking + bottomhalf + tracking_bottomhalf', data=df).fit()
hat_gamma3=model_clustered3.params[1]
hat_se3=model_clustered3.bse[1]
print(model_clustered3.summary())

                            OLS Regression Results                            
Dep. Variable:             totalscore   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     335.9
Date:                Wed, 11 Jun 2025   Prob (F-statistic):          1.13e-199
Time:                        22:38:51   Log-Likelihood:                -18751.
No. Observations:                5304   AIC:                         3.751e+04
Df Residuals:                    5300   BIC:                         3.754e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              15.4869    

C:\Users\Camil\AppData\Local\Temp\ipykernel_3004\2807184409.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_gamma3=model_clustered3.params[1]
C:\Users\Camil\AppData\Local\Temp\ipykernel_3004\2807184409.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se3=model_clustered3.bse[1]


## Challenge 3: Inferencia errónea por no agrupar
Compare los errores estándar de $\hat{\gamma}$ usando:
- Errores convencionales.
- Errores robustos de White (HC0)
- Errores robustos agrupados por escuela.


¿En cuál caso cambia la significancia estadística del efecto de tracking?